### This is a documents on how to implement a simple program to solve the unrestricted Hartree-Fock with SCF iterations based on STO-3g.

# Dependencies
- `python` 3.9.19 not necessary
- `numpy` 1.26.4
- `scipy` 1.10.1

# Unrestricted Hartree-Fock
UHF equation

\begin{align}
{\bf F}^{\alpha}{\bf C}^{\alpha} &= {\bf SC}^{\alpha}{\bf\epsilon}^{\alpha} \\
{\bf F}^{\beta}{\bf C}^{\beta} &= {\bf SC}^{\beta}{\bf\epsilon}^{\beta},
\end{align}

The one-electron Fock matrices are given by

\begin{align}
F_{\mu\nu}^{\alpha} &= H_{\mu\nu} + (\mu\,\nu\mid\lambda\,\sigma)[D_{\lambda\sigma}^{\alpha} + D_{\lambda\sigma}^{\beta}] - (\mu\,\lambda\,\mid\nu\,\sigma)D_{\lambda\sigma}^{\alpha}\\
F_{\mu\nu}^{\beta} &= H_{\mu\nu} + (\mu\,\nu\mid\,\lambda\,\sigma)[D_{\lambda\sigma}^{\alpha} + D_{\lambda\sigma}^{\beta}] - (\mu\,\lambda\,\mid\nu\,\sigma)D_{\lambda\sigma}^{\beta},
\end{align}

where the density matrices $D_{\lambda\sigma}^{\alpha}$ and $D_{\lambda\sigma}^{\beta}$ are given by

\begin{align}
D_{\lambda\sigma}^{\alpha} &= C_{\sigma i}^{\alpha}C_{\lambda i}^{\alpha}\\
D_{\lambda\sigma}^{\beta} &= C_{\sigma i}^{\beta}C_{\lambda i}^{\beta}.
\end{align}

Unlike for RHF, the orbital coefficient matrices ${\bf C}^{\alpha}$ and ${\bf C}^{\beta}$ are of dimension $M\times N^{\alpha}$ and $M\times N^{\beta}$, where $M$ is the number of AO basis functions and $N^{\alpha}$ ($N^{\beta}$) is the number of $\alpha$ ($\beta$) electrons.  The total UHF energy is given by

\begin{align}
E^{\rm UHF}_{\rm total} &= E^{\rm UHF}_{\rm elec} + E^{\rm BO}_{\rm nuc},\;\;{\rm with}\\
E^{\rm UHF}_{\rm elec} &= \frac{1}{2}[({\bf D}^{\alpha} + {\bf D}^{\beta}){\bf H} + 
{\bf D}^{\alpha}{\bf F}^{\alpha} + {\bf D}^{\beta}{\bf F}^{\beta}].
\end{align}

The following is programming implementation of UHF:

In [6]:
#########################==> PART1 <==#########################
'''
    construct STO-3g basis for H & C atoms
'''
import numpy as np
import numpy.linalg as la
from scipy.special import factorial as fac
from scipy.special import factorial2 as fac2
from scipy.special import comb
import scipy.special as spec


## Part1
#### Define Class Atom for H & C

#### Store exponential coefficients for the Gaussian orbitals

In [7]:
class Atom:
    #Atom class -- H atom & C atom

    def __init__(self,name,Z,orb):
        """
            atom_name: H or C
            Z: Atomic number in periodic table
            orb: orbitals for this atom
        """

        self.name = name
        self.orb= orb
        self.Z = Z

class STO3G():

    def __init__(self,atom):
        # Exponential coefficients for the Gaussian orbitals
        self.zeta_o1 = {"H":1.24,
                        "C":5.67}
        self.zeta_o2 = {"C":1.72}

        self.STO3G = []

        for orbiral in atom.orb:
            if orbiral == "1s":
                a1 = 0.109818 * self.zeta_o1[atom.name]**2
                a2 = 0.405771 * self.zeta_o1[atom.name]**2
                a3 = 2.22766 * self.zeta_o1[atom.name]**2
                d1 = 0.444635
                d2 = 0.535328
                d3 = 0.154329

                self.STO3G.append({  "AOn":atom.name,
                                "AOt":orbiral,
                                "R":(0,0,0),
                                "lx":0,
                                "ly":0,
                                "lz":0,
                                "a":(a1,a2,a3),
                                "d":(d1,d2,d3)})

            if orbiral == "2s":
                a1 = 0.0751386 * self.zeta_o2[atom.name]**2
                a2 = 0.231031 * self.zeta_o2[atom.name]**2
                a3 = 0.994203 * self.zeta_o2[atom.name]**2
                d1 = 0.700115
                d2 = 0.399513
                d3 = -0.0999672

                self.STO3G.append({  "AOn":atom.name,
                            "AOt":orbiral,
                            "R":(0,0,0),
                            "lx":0,
                            "ly":0,
                            "lz":0,
                            "a":(a1,a2,a3),
                            "d":(d1,d2,d3)})

            if orbiral == "2p":
                a1 = 0.0751386 * self.zeta_o2[atom.name]**2
                a2 = 0.231031 * self.zeta_o2[atom.name]**2
                a3 = 0.994203 * self.zeta_o2[atom.name]**2
                d1 = 0.391957
                d2 = 0.607684
                d3 = 0.1559163

                self.STO3G.append({  "AOn":atom.name,
                                "AOt":orbiral,
                                "R":(0,0,0),
                                "lx":1,
                                "ly":0,
                                "lz":0,
                                "a":(a1,a2,a3),
                                "d":(d1,d2,d3)})
                self.STO3G.append({  "AOn":atom.name,
                                "AOt":orbiral,
                                "R":(0,0,0),
                                "lx":0,
                                "ly":1,
                                "lz":0,
                                "a":(a1,a2,a3),
                                "d":(d1,d2,d3)})
                self.STO3G.append({  "AOn":atom.name,
                                "AOt":orbiral,
                                "R":(0,0,0),
                                "lx":0,
                                "ly":0,
                                "lz":1,
                                "a":(a1,a2,a3),
                                "d":(d1,d2,d3)})

        self.length = len(self.STO3G)

    def basis(self):
        return self.STO3G

    
    def print(self):

        for b in self.STO3G:
            print(b["AOt"] + ":","lx = " + str(b["lx"]),"ly = " + str(b["ly"]),"lz = " + str(b["lz"]))
            print("      alpha = ", b["a"])
            print("      d = ", b["d"],'\n')

#### The following part is test area of part1.

#### We initialize a hydrogen atom and print its sto-3g basis information

In [8]:
#Test basis region

if __name__ == "__main__":
    atoms = Atom("C",6,["1s","2s","2p"])

    # Create the basis set
    sto3g = STO3G(atoms)
    print(sto3g)
    sto3g.print()

1s: lx = 0 ly = 0 lz = 0
      alpha =  (3.5305279001999996, 13.0450913019, 71.616818574)
      d =  (0.444635, 0.535328, 0.154329) 

2s: lx = 0 ly = 0 lz = 0
      alpha =  (0.22229003424, 0.6834821103999998, 2.9412501551999997)
      d =  (0.700115, 0.399513, -0.0999672) 

2p: lx = 1 ly = 0 lz = 0
      alpha =  (0.22229003424, 0.6834821103999998, 2.9412501551999997)
      d =  (0.391957, 0.607684, 0.1559163) 

2p: lx = 0 ly = 1 lz = 0
      alpha =  (0.22229003424, 0.6834821103999998, 2.9412501551999997)
      d =  (0.391957, 0.607684, 0.1559163) 

2p: lx = 0 ly = 0 lz = 1
      alpha =  (0.22229003424, 0.6834821103999998, 2.9412501551999997)
      d =  (0.391957, 0.607684, 0.1559163) 



## Part2
#### Calculate Gaussian integral to get:

    kinetic integral between two Cartesian gaussian functions

    uclear-electron interaction integrals

    electron-electron interaction integrals

In [9]:
#########################==> PART2 <==#########################
'''
    various integrals
    1.prodOfGauss: Gaussian produc theorem
    2.Overxyz: Overlap of the two gaussians along the chosen direction
    3.overlap: Overlap of the two Gaussian
    4.kinetic: kinetic integral between two Cartesian gaussian functions
    5.nuclear: uclear-electron interaction integrals
    6.elecronic: electron-electron interaction integrals
    
'''
##################################################
def normalFac(lx,ly,lz,ec):
    # normalized coefficient
    N = (2*ec/np.pi)**(3/4)
    N *= (4*ec)**((lx+ly+lz)/2)
    N /= np.sqrt( fac2(2*lx-1) * fac2(2*ly-1) * fac2(2*lz-1) )

    return N

    
def prodOfGauss(Eg1,Eg2,Cg1,Cg2):

    #Eg1: Exponential coefficient of Gaussian 1
    #Eg2: Exponential coefficient of Gaussian 2
    #Cg1: Center of Gaussian 1
    #Cg2: Center of Gaussian 2

    #R: produc center
    #c: produc coefficient

    Cg1 = np.asarray(Cg1)
    Cg2 = np.asarray(Cg2)

    R = (Eg1 * Cg1 + Eg2 * Cg2) / (Eg1 + Eg2)

    c = np.dot(Cg1-Cg2,Cg1-Cg2)
    c *= - Eg1*Eg2 / (Eg1 + Eg2)
    c = np.exp(c)

    return R,c



def Overxyz(lg1,lg2,Eg1,Eg2,Cg1,Cg2,R):
    """
        lg1: Angular momentum along the chosen direction for Gaussian 1
        lg2: Angular momentum along the chosen direction for Gaussian 2
        others are same as function prodOfGauss(Eg1,Eg2,Cg1,Cg2)

        S: Overlap of the two gaussians along the chosen direction
    """

    ove = 0

    for i in range(lg1+1):
        for j in range(lg2+1):
            if (i+j) % 2 == 0:
                tmp = comb(lg1,i,exact=True)
                tmp *= comb(lg2,j,exact=True)
                tmp *= fac2(i + j - 1,exact=True)
                tmp /= (2.*(Eg1+Eg2))**((i+j)/2.)
                tmp *= (R-Cg1)**(lg1-i)
                tmp *= (R-Cg2)**(lg2-j)

                ove += tmp

    return ove
##################################################

def overlap(lgx1,lgy1,lgz1,lgx2,lgy2,lgz2,Eg1,Eg2,Cg1,Cg2):
    #overlapOfGauss: Overlap of the two Gaussian

    R,c = prodOfGauss(Eg1,Eg2,Cg1,Cg2)
    Na = normalFac(lgx1,lgy1,lgz1,Eg1)
    Nb = normalFac(lgx2,lgy2,lgz2,Eg2)

    overlapOfGauss = 1
    overlapOfGauss *= Overxyz(lgx1,lgx2,Eg1,Eg2,Cg1[0],Cg2[0],R[0]) # Overlap along x
    overlapOfGauss *= Overxyz(lgy1,lgy2,Eg1,Eg2,Cg1[1],Cg2[1],R[1]) # Overlap along y
    overlapOfGauss *= Overxyz(lgz1,lgz2,Eg1,Eg2,Cg1[2],Cg2[2],R[2]) # Overlap along z
    overlapOfGauss *= Na * Nb * c # Product coefficient and normalization
    overlapOfGauss *= (np.pi / (Eg1+Eg2))**(3./2.) # Normalization

    return overlapOfGauss


def kinetic(lgx1,lgy1,lgz1,lgx2,lgy2,lgz2,Eg1,Eg2,Cg1,Cg2):

    R,c = prodOfGauss(Eg1,Eg2,Cg1,Cg2)

    def Kxyz(ac,a1,a2,bc,b1,b2,aa,bb,Ra,Rb,Ra1,Rb1,Ra2,Rb2,Rc,R1,R2):
        """
                KC: Kinetic integral between two gaussians along chosen direction
        """

        kc = 0
        kc += ac * bc * Overxyz(ac-1,bc-1,aa,bb,Ra,Rb,Rc)
        kc += -2 * aa * bc * Overxyz(ac+1,bc-1,aa,bb,Ra,Rb,Rc)
        kc += -2 * ac * bb * Overxyz(ac-1,bc+1,aa,bb,Ra,Rb,Rc)
        kc += 4 * aa * bb * Overxyz(ac+1,bc+1,aa,bb,Ra,Rb,Rc)
        kc *= 0.5

        Kc = 1
        Kc *= c * (np.pi / (aa+bb))**(3./2.) * kc
        Kc *= Overxyz(a1,b1,aa,bb,Ra1,Rb1,R1)
        Kc *= Overxyz(a2,b2,aa,bb,Ra2,Rb2,R2)

        return Kc

    # Cyclic permutation of the entries
    Kx = Kxyz(lgx1,lgy1,lgz1,lgx2,lgy2,lgz2,Eg1,Eg2,Cg1[0],Cg2[0],Cg1[1],Cg2[1],Cg1[2],Cg2[2],R[0],R[1],R[2]) # Kinetic integral along x
    Ky = Kxyz(lgy1,lgz1,lgx1,lgy2,lgz2,lgx2,Eg1,Eg2,Cg1[1],Cg2[1],Cg1[2],Cg2[2],Cg1[0],Cg2[0],R[1],R[2],R[0]) # Kinetic integral along y
    Kz = Kxyz(lgz1,lgx1,lgy1,lgz2,lgx2,lgy2,Eg1,Eg2,Cg1[2],Cg2[2],Cg1[0],Cg2[0],Cg1[1],Cg2[1],R[2],R[0],R[1]) # Kinetic integral along z

    Na = normalFac(lgx1,lgy1,lgz1,Eg1) 
    Nb = normalFac(lgx2,lgy2,lgz2,Eg2) 

    K = (Kx + Ky + Kz) * Na * Nb 

    return K

##################################################
def f(j,l,m,a,b):
    f = 0

    for k in range(max(0,j-m),min(j,l)+1):
        tmp = 1
        tmp *= spec.binom(l,k)
        tmp *= spec.binom(m,j-k)
        tmp *= a**(l-k)
        tmp *= b**(m+k-j)

        f += tmp

    return f

def F(nu,x):
    """
    Boys function.
        nu: Boys function index
        x: Boys function variable
        ff: Value of the Boys function for index NU evaluated at X
    """
    if x < 1e-8:
        ff =  1 / (2 * nu + 1) - x / (2 * nu + 3)
    else:
        ff = 0.5 / x**(nu+0.5) * spec.gamma(nu+0.5)*spec.gammainc(nu+0.5,x)

    return ff
##################################################

def nuclear(ax,ay,az,bx,by,bz,aa,bb,Ra,Rb,Rn,Zn):
    """
    Compute nuclear-electron interaction integrals.
    """
    Vn = 0

    # Intermediate variable
    g = aa + bb
    eps = 1. / (4 * g)

    Rp,c = prodOfGauss(aa,bb,Ra,Rb) # Gaussian product

    def A(l,r,i,l1,l2,Cg1,Cg2,Cg3,Cg4):

        #Expansion coefficient A.

        A = 1
        A *= (-1)**(l)
        A *= f(l,l1,l2,Cg4-Cg1,Cg4-Cg2)
        A *= (-1)**i
        A *= fac(l,exact=True)
        A *= (Cg4-Cg3)**(l-2*r-2*i)
        A *= eps**(r+i)
        A /= fac(r,exact=True)
        A /= fac(i,exact=True)
        A /= fac(l-2*r-2*i,exact=True)

        return A

    for l in range(0,ax+bx+1):
        for r in range(0,int(l/2)+1):
            for i in range(0,int((l-2*r)/2)+1):
                Ax = A(l,r,i,ax,bx,Ra[0],Rb[0],Rn[0],Rp[0])

                for m in range(0,ay+by+1):
                    for s in range(0,int(m/2)+1):
                        for j in range(0,int((m-2*s)/2)+1):
                            Ay = A(m,s,j,ay,by,Ra[1],Rb[1],Rn[1],Rp[1])

                            for n in range(0,az+bz+1):
                                for t in range(0,int(n/2)+1):
                                    for k in range(0,int((n-2*t)/2)+1):
                                        Az =  A(n,t,k,az,bz,Ra[2],Rb[2],Rn[2],Rp[2])

                                        nu = l + m + n - 2 * (r + s + t) - (i + j + k) # Index of Boys function

                                        ff = F(nu,g*np.dot(Rp-Rn,Rp-Rn)) # Boys function

                                        Vn += Ax * Ay * Az * ff

    # Compute normalization
    Na = normalFac(ax,ay,az,aa)
    Nb = normalFac(bx,by,bz,bb)

    Vn *= - Zn * Na * Nb * c * 2 * np.pi / g

    return Vn


def electronic(ax,ay,az,bx,by,bz,cx,cy,cz,dx,dy,dz,aa,bb,cc,dd,Ra,Rb,Rc,Rd):
    """
        G: Electron-electron integral
    """

    G = 0

    g1 = aa + bb
    g2 = cc + dd

    Rp, c1 = prodOfGauss(aa,bb,Ra,Rb)
    Rq, c2 = prodOfGauss(cc,dd,Rc,Rd)

    delta = 1 / (4 * g1) + 1 / (4 * g2)

    def theta(l,l1,l2,a,b,r,g):
        #Expansion coefficient theta.
        t = 1
        t *= f(l,l1,l2,a,b)
        t *= fac(l,exact=True)
        t *= g**(r-l)
        t /= fac(r,exact=True) * fac(l-2*r,exact=True)

        return t


    def B(l,ll,r,rr,i,l1,l2,Ra,Rb,Rp,g1,l3,l4,Rc,Rd,Rq,g2):

        #Expansion coefficient B.

        b = 1
        b *= (-1)**(l) * theta(l,l1,l2,Rp-Ra,Rp-Rb,r,g1)
        b *= theta(ll,l3,l4,Rq-Rc,Rq-Rd,rr,g2)
        b *= (-1)**i * (2*delta)**(2*(r+rr))
        b *= fac(l + ll - 2*r - 2*rr,exact=True)
        b *= delta**i * (Rp-Rq)**(l+ll-2*(r+rr+i))

        tmp = 1
        tmp *= (4*delta)**(l+ll) * fac(i,exact=True)
        tmp *= fac(l+ll-2*(r+rr+i),exact=True)

        b /= tmp

        return b

    for l in range(0,ax+bx+1):
        for r in range(0,int(l/2)+1):
            for ll in range(0,cx+dx+1):
                for rr in range(0,int(ll/2)+1):
                    for i in range(0,int((l+ll-2*r-2*rr)/2)+1):
                        Bx = B(l,ll,r,rr,i,ax,bx,Ra[0],Rb[0],Rp[0],g1,cx,dx,Rc[0],Rd[0],Rq[0],g2)

                        for m in range(0,ay+by+1):
                            for s in range(0,int(m/2)+1):
                                for mm in range(0,cy+dy+1):
                                    for ss in range(0,int(mm/2)+1):
                                        for j in range(0,int((m+mm-2*s-2*ss)/2)+1):
                                            By = B(m,mm,s,ss,j,ay,by,Ra[1],Rb[1],Rp[1],g1,cy,dy,Rc[1],Rd[1],Rq[1],g2)

                                            for n in range(0,az+bz+1):
                                                for t in range(0,int(n/2)+1):
                                                    for nn in range(0,cz+dz+1):
                                                        for tt in range(0,int(nn/2)+1):
                                                            for k in range(0,int((n+nn-2*t-2*tt)/2)+1):
                                                                Bz = B(n,nn,t,tt,k,az,bz,Ra[2],Rb[2],Rp[2],g1,cz,dz,Rc[2],Rd[2],Rq[2],g2)

                                                                nu = l+ll+m+mm+n+nn-2*(r+rr+s+ss+t+tt) - (i + j + k)

                                                                ff = F(nu,np.dot(Rp-Rq,Rp-Rq)/(4.*delta))

                                                                G += Bx * By * Bz * ff


    # Compute normalization
    Na = normalFac(ax,ay,az,aa)
    Nb = normalFac(bx,by,bz,bb)
    Nc = normalFac(cx,cy,cz,cc)
    Nd = normalFac(dx,dy,dz,dd)

    G *= Na * Nb * Nc * Nd * c1 * c2 * 2 * np.pi**2 / (g1 * g2) * np.sqrt(np.pi / (g1 + g2))

    return G


def EE_list(basis):

    #EE: list of two-electron integrals, with indices (i,j,k,l)

    # Size of the basis set
    K = basis.length

    # List of basis functions
    B = basis.basis()

    EE = np.zeros((K,K,K,K))

    Nee= 0

    for i,b1 in enumerate(B):
        for j,b2 in enumerate(B):
            for k,b3 in enumerate(B):
                for l,b4 in enumerate(B):

                    Nee += 1

                    # Print update of calculation (can be long)
                    if Nee % 500 == 0:
                        print("     Computed ", Nee, " two-electron integrals of ", K**4, ".",sep='')


                    for a1,d1 in zip(b1["a"],b1["d"]):
                        for a2,d2 in zip(b2["a"],b2["d"]):
                            for a3,d3 in zip(b3["a"],b3["d"]):
                                for a4,d4 in zip(b4["a"],b4["d"]):
                                    # Basis functions centers
                                    R1 = b1["R"]
                                    R2 = b2["R"]
                                    R3 = b3["R"]
                                    R4 = b4["R"]

                                    # Basis functions angular momenta
                                    ax = b1["lx"]
                                    ay = b1["ly"]
                                    az = b1["lz"]

                                    # Basis functions angular momenta
                                    bx = b2["lx"]
                                    by = b2["ly"]
                                    bz = b2["lz"]

                                    # Basis functions angular momenta
                                    cx = b3["lx"]
                                    cy = b3["ly"]
                                    cz = b3["lz"]

                                    # Basis functions angular momenta
                                    dx = b4["lx"]
                                    dy = b4["ly"]
                                    dz = b4["lz"]

                                    tmp = 1
                                    tmp *= d1.conjugate()*d2.conjugate()
                                    tmp *= d3 * d4
                                    tmp *= electronic(ax,ay,az,bx,by,bz,cx,cy,cz,dx,dy,dz,a1,a2,a3,a4,R1,R2,R3,R4)

                                    EE[i,j,k,l] += tmp

    return EE

def print_EE_list(ee):
    """
    Print list of electron-electron integrals.

    """

    K = ee.shape[0]

    for i in range(K):
        for j in range(K):
            for k in range(K):
                for l in range(K):
                    print("({0},{1},{2},{3})  {4}".format(i+1,j+1,k+1,l+1,ee[i,j,k,l]))




#### The following part is test area of part2.

#### We initialize a hydrogen atom and print its e-e integral

In [10]:
# check integral region
if __name__ == "__main__":
    H = Atom("H",1,["1s"])

    sto3g_H = STO3G(H)
    sto3g_H.print()
    ee_H = EE_list(sto3g_H) 
    print_EE_list(ee_H)


1s: lx = 0 ly = 0 lz = 0
      alpha =  (0.16885615680000002, 0.6239134896, 3.4252500160000006)
      d =  (0.444635, 0.535328, 0.154329) 

(1,1,1,1)  0.7746083600328784


## Part3
#### Calculate Matirx:

    1.canonical orthogonalization Matrix to diag F'

    2.H_core Matrix = T + V

    3.e-e Matrix

In [11]:
#########################==> PART3 <==#########################
def M_overlap(basis):


    K = basis.length
    B = basis.basis()
    M_o = np.zeros((K,K))

    for i,b1 in enumerate(B):
        for j,b2 in enumerate(B):
            for a1,d1 in zip(b1["a"],b1["d"]):
                for a2,d2 in zip(b2["a"],b2["d"]):
                    R1 = b1["R"]
                    R2 = b2["R"]

                    tmp = d1.conjugate()*d2
                    tmp *= overlap(b1["lx"],b1["ly"],b1["lz"],b2["lx"],b2["ly"],b2["lz"],a1,a2,R1,R2)

                    M_o[i,j] +=  tmp

    return M_o


def S_p(S):
    #canonical orthogonalization.

    s, U = la.eig(S)
    s = np.diag(s**(-1./2.))
    T = np.dot(U,s)

    return T


def M_kinetic(basis):
    K = basis.length
    B = basis.basis()
    T = np.zeros((K,K))

    for i,b1 in enumerate(B):
        for j,b2 in enumerate(B):
            for a1,d1 in zip(b1["a"],b1["d"]):
                for a2,d2 in zip(b2["a"],b2["d"]):
                    R1 = b1["R"]
                    R2 = b2["R"]

                    tmp = d1.conjugate()*d2
                    tmp *= kinetic(b1["lx"],b1["ly"],b1["lz"],b2["lx"],b2["ly"],b2["lz"],a1,a2,R1,R2)

                    T[i,j] +=  tmp

    return T


def M_nuclear(basis,atom):

    #nuclear-electron potential
    K = basis.length
    B = basis.basis()
    Rn = (0,0,0)
    Zn = atom.Z
    Vn = np.zeros((K,K))

    for i,b1 in enumerate(B):
        for j,b2 in enumerate(B):
            for a1,d1 in zip(b1["a"],b1["d"]):
                for a2,d2 in zip(b2["a"],b2["d"]):
                    R1 = b1["R"]
                    R2 = b2["R"]

                    tmp = d1.conjugate()*d2
                    tmp *= nuclear(b1["lx"],b1["ly"],b1["lz"],b2["lx"],b2["ly"],b2["lz"],a1,a2,R1,R2,Rn,Zn)

                    Vn[i,j] +=  tmp

    return Vn


def H_core(basis,atom):
    #core Hamiltonian

    T = M_kinetic(basis)
    V=M_nuclear(basis,atom)
    return T + V


def P_density(C,norb):
    """

        C: Matrix of coefficients
        N: Number of electrons
    """
 

    K = C.shape[0]
    P = np.zeros((K,K))
    C_occ=C[:,:norb]
    P=np.einsum('pi,qi->pq',C_occ,C_occ,optimize=True)

    return P



def M_ee(basis,Palpha,Pbeta,ee):

    #Electron-electron interaction matrix
    K = basis.length
    Ga= np.zeros((K,K))
    Gb= np.zeros((K,K))
    for i in range(K):
        for j in range(K):
            for k in range(K):
                for l in range(K):
                    Ga[i,j] += (Palpha[k,l]+Pbeta[k,l]) * (ee[i,j,k,l])  -Palpha[k,l] * ee[i,l,k,j]
                    Gb[i,j] += (Palpha[k,l]+Pbeta[k,l]) * (ee[i,j,k,l])  -Pbeta[k,l] * ee[i,l,k,j]
    return Ga,Gb

#### The following part is test area of part3.

#### We initialize a hydrogen atom and print its matrices

In [12]:
H = Atom("H",1,["1s"])
sto3g_H = STO3G(H)

# Compute matrices
S_H = M_overlap(sto3g_H)
T_H = M_kinetic(sto3g_H)
Vn_H = M_nuclear(sto3g_H,H)
H_core_H = H_core(sto3g_H,H)
print(S_H)
print(T_H)
print(Vn_H)
print(H_core_H)

[[1.00000143]]
[[0.76003294]]
[[-1.22661547]]
[[-0.46658252]]


## Now, let's do SCF calculation to get the ground state energy and single-particle orbital energy of atom

### The SCF procedure

1. Obtain a guess at the density matrix.
2. Calculate the exchange and coulomb matrices from the density matrix and the two-electron repulsion integrals.
3. Add exchange and coulomb matrices to the core-Hamiltonian to obtain the Fock matrix.
4. Diagonalize the Fock matrix.
5. Select the occupied orbitals and calculate the new density matrix.
6. Compute the energy.
7. Compute the errors and check for convergence.

In [13]:
def UHF_step(basis,atom,Na,Nb,H,X,Pa,Pb,ee):
    """
        basis: basis
        atom
        Na: Number of alpha electrons
        Nb: Number of beta electrons
        H: core Hamiltonian
        X: orthnormal matrix
        Pa: alpha density matrix
        Pb: beta density matrix
        ee: e-e Integrals
    """

    Ga,Gb=M_ee(basis,Pa,Pb,ee)
    Fa = H + Ga
    Fb = H + Gb

    Fax = np.dot(X.conj().T,np.dot(Fa,X)) # Compute Fock matrix in the orthonormal basis set (S=I in this set)
    Fbx = np.dot(X.conj().T,np.dot(Fb,X))

    ea, Cxa = la.eigh(Fax) # Compute eigenvalues and eigenvectors of the Fock matrix
    eb, Cxb = la.eigh(Fbx) # Compute eigenvalues and eigenvectors of the Fock matrix

    idx1 = ea.argsort()
    ea= ea[idx1]
    Cxa = Cxa[:,idx1]

    idx2 = eb.argsort()
    eb = eb[idx2]
    Cxb = Cxb[:,idx2]

    ea = np.diag(ea) # Extract orbital energies as vector
    eb = np.diag(eb) # Extract orbital energies as vector


    Ca = np.dot(X,Cxa) # Transform coefficient matrix in the orthonormal basis to the original basis
    Cb = np.dot(X,Cxb)

    Panew = P_density(Ca,Na) # Compute the new density matrix
    Pbnew = P_density(Cb,Nb) # Compute the new density matrix

    return Panew, Fa, ea, Pbnew, Fb, eb

def delta_P(P_old,P_new):
    delta = 0

    n = P_old.shape[0]

    for i in range(n):
        for j in range(n):
            delta += (P_old[i,j] - P_new[i,j])**2

    return (delta / 4.)**(0.5)

def escf(Pa,Pb,Fa,Fb,H):
    
    Escf = 0
    Escf = np.einsum('pq,pq->', (Pa + Pb), H, optimize=True)
    Escf += np.einsum('pq,pq->', Pa, Fa, optimize=True)
    Escf += np.einsum('pq,pq->', Pb, Fb, optimize=True)
    Escf *= 0.5
    
    return Escf

## Carbon Atom
![title](Catom.png)

## 4 alpha electons, 2 beta electrons

In [17]:
# H = Atom("H",1,["1s"])
# sto3g_H = STO3G(H)
H = Atom("C",6,["1s","2s","2p"])
sto3g_H = STO3G(H)
mol=H
bs=sto3g_H
Na = 4 # Number of alpha electrons
Nb = 2 

maxiter = 100 # Maximal number of iteration
verbose= True
# Basis set size
K = bs.length

print("Computing overlap matrix S...")
S = M_overlap(bs)

if verbose:
    print(S)

print("Computing orthogonalization matrix X...")
X = S_p(S)

if verbose:
    print(X)

print("Computing core Hamiltonian...")
Hc = H_core(bs,mol)

if verbose:
    print(Hc)

#print("Computing two-electron integrals...")
ee = EE_list(bs)

#if verbose:
#    print_EE_list(ee)

Panew = np.zeros((K,K))
Pa = np.zeros((K,K))
Pbnew = np.zeros((K,K))
Pb = np.zeros((K,K))

converged = False

iter = 1
while not converged and iter <= maxiter:
    print("\n\n\n#####\nSCF cycle " + str(iter) + ":")
    print("#####")

    Panew, Fa, ea, Pbnew, Fb, eb = UHF_step(bs,mol,Na,Nb,Hc,X,Pa,Pb,ee)

    # Print results of the SCF step
    print("\nTotal energy:", escf(Pa,Pb,Fa,Fb,Hc),"\n")
    print("   Orbital energies:")
    print("   Alpha electrons:")
    print("   ", np.diag(ea),'\n')
    print("   Beta electrons:")
    print("   ", np.diag(eb),'\n')
    

    # Check convergence of the SCF cycle
    if delta_P(Pa,Panew) < 1e-20:
        converged = True

        print("\n\n\nTOTAL ENERGY:", escf(Pa,Pb,Fa,Fb,Hc)) # Print final, total energy

    if iter == maxiter:
        print("SCF NOT CONVERGED!")

    Pa = Panew
    Pb = Pbnew

    iter += 1

Computing overlap matrix S...
[[1.00000143 0.24836194 0.         0.         0.        ]
 [0.24836194 0.99999979 0.         0.         0.        ]
 [0.         0.         1.00000019 0.         0.        ]
 [0.         0.         0.         1.00000019 0.        ]
 [0.         0.         0.         0.         1.00000019]]
Computing orthogonalization matrix X...
[[ 0.63287123 -0.81560472  0.          0.          0.        ]
 [ 0.63286914  0.81560741  0.          0.          0.        ]
 [ 0.          0.          0.9999999   0.          0.        ]
 [ 0.          0.          0.          0.9999999   0.        ]
 [ 0.          0.          0.          0.          0.9999999 ]]
Computing core Hamiltonian...
[[-17.76164483  -4.27811749   0.           0.           0.        ]
 [ -4.27811749  -4.71751527   0.           0.           0.        ]
 [  0.           0.          -3.67049263   0.           0.        ]
 [  0.           0.           0.          -3.67049263   0.        ]
 [  0.           0.  

## Hydrogen Atom

In [17]:
H = Atom("H",1,["1s"])
sto3g_H = STO3G(H)
mol=H
bs=sto3g_H
Na = 1 # Number of alpha electrons
Nb = 0 

maxiter = 100 # Maximal number of iteration
verbose= True
# Basis set size
K = bs.length

print("Computing overlap matrix S...")
S = M_overlap(bs)

if verbose:
    print(S)

print("Computing orthogonalization matrix X...")
X = S_p(S)

if verbose:
    print(X)

print("Computing core Hamiltonian...")
Hc = H_core(bs,mol)

if verbose:
    print(Hc)

print("Computing two-electron integrals...")
ee = EE_list(bs)

if verbose:
    print_EE_list(ee)

Panew = np.zeros((K,K))
Pa = np.zeros((K,K))
Pbnew = np.zeros((K,K))
Pb = np.zeros((K,K))

converged = False

iter = 1
while not converged and iter <= maxiter:
    print("\n\n\n#####\nSCF cycle " + str(iter) + ":")
    print("#####")

    Panew, Fa, ea, Pbnew, Fb, eb = UHF_step(bs,mol,Na,Nb,Hc,X,Pa,Pb,ee)

    # Print results of the SCF step
    print("\nTotal energy:", escf(Pa,Pb,Fa,Fb,Hc),"\n")
    print("   Orbital energies:")
    print("   Alpha electrons:")
    print("   ", np.diag(ea),'\n')
    print("   Beta electrons:")
    print("   ", np.diag(eb),'\n')
    

    # Check convergence of the SCF cycle
    if delta_P(Pa,Panew) < 1e-20:
        converged = True

        print("\n\n\nTOTAL ENERGY:", escf(Pa,Pb,Fa,Fb,Hc)) # Print final, total energy

    if iter == maxiter:
        print("SCF NOT CONVERGED!")

    Pa = Panew
    Pb = Pbnew

    iter += 1

Computing overlap matrix S...
[[1.00000143]]
Computing orthogonalization matrix X...
[[0.99999929]]
Computing core Hamiltonian...
[[-0.46658252]]
Computing two-electron integrals...
(1,1,1,1)  0.7746083600328784



#####
SCF cycle 1:
#####

Total energy: 0.0 

   Orbital energies:
   Alpha electrons:
    [-0.46658186] 

   Beta electrons:
    [-0.46658186] 




#####
SCF cycle 2:
#####

Total energy: -0.4665818591484326 

   Orbital energies:
   Alpha electrons:
    [-0.46658186] 

   Beta electrons:
    [0.30802429] 




TOTAL ENERGY: -0.4665818591484326
